In [ ]:
import pandas as pd
import re
from neo4j import GraphDatabase
from datetime import datetime

class add_node:

    def __init__(self, filepath, uri, user, password):
        self.filepath = filepath
        self.uri = uri
        self.user = user
        self.password = password
        self.driver = GraphDatabase.driver(uri, auth=(user, password))


    def delete_graph(self):
        driver = self.driver
        
        with driver.session() as session:
            session.run("MATCH (n) DETACH DELETE n;")
        driver.close()

    def _extract_acronym(self,text):

        match = re.search(r"\((.*?)\)", text)  # Find text inside parentheses

        if match:
            acronym = match.group(1)  # Extract AMS
            return acronym
        
        else:
            return text
        
    
    def standardize_date(self, date_str):
        
        fmt = "%m/%d/%Y"
        print(type(date_str))
        try:
            if isinstance(date_str, str):
                return datetime.strptime(date_str, fmt).strftime("%Y-%m-%d")  # Convert to YYYY-MM-DD
            else:
                return date_str
        except ValueError:
            # continue  # Try the next format if parsing fails
            return None  # Return None if no format matches

    def getting_titles(self, dfe, start_row, start_column):
    # Locate relevant data based on structure

        title = dfe.iloc[start_row, start_column]  # Assuming position title is at row 4, column 0
        num_positions = dfe.iloc[start_row, start_column+1]  # Assuming number of positions at row 4, column 1
        num_accounts = dfe.iloc[start_row, start_column+2]  # Assuming number of accounts at row 4, column 2
        
        if pd.notna(title) and pd.notna(num_positions) and pd.notna(num_accounts):
        
        
            pos_data = {
                "title": title,
                "num_positions": num_positions,
                "num_accounts": num_accounts
            }

            return [pos_data] + self.getting_titles(dfe, start_row+1, start_column)
        
        else:

            return []

    def extract_data(self):
        file_path  = self.filepath
        xlsx = pd.read_excel(file_path, sheet_name=None)  # Load all sheets into a dictionary
        
        # Extract agency name
        general_info_df = xlsx['General_Information']
        agency_name = general_info_df.iloc[5, 1]  # Assuming 'Name of Agency' is at row 5, column 1
        
        # Extract totals
        totals_df = xlsx['TOTALS']
        total_positions = totals_df.iloc[13, 1]  # Assuming total positions is at row 13, column 1
        total_accounts = totals_df.iloc[13, 2]   # Assuming total accounts is at row 13, column 2
        total_categories = 10  # Hardcoded based on category sheet names
        
        # Extract positions from category sheets
        category_sheets = [sheet for sheet in xlsx.keys() if sheet.startswith("Category_")]
        positions = []
        
        for sheet in category_sheets:
            df = xlsx[sheet]
            
            # Locate relevant data based on structure
            try:
                pos_from_one_tab = self.getting_titles(df, 4, 0)
                positions.extend(pos_from_one_tab)
            except:
                continue
        
        # Create agency node
        agency_node = {
            "name": agency_name,
            "total_positions": total_positions,
            "total_accounts": total_accounts,
            "total_categories": total_categories
        }
        
        return agency_node, positions
    
    # Function to create nodes and relationships in Neo4j
    def create_agency_with_positions(self, agency, positions):
        driver = self.driver

        with driver.session() as session:
            # Create Agency node
            session.run(
                """
                MERGE (a:Agency {name: $name})
                SET a.TotalPositions = $total_positions,
                    a.TotalAccounts = $total_accounts,
                    a.TotalCategories = $total_categories
                """,
                name=agency["name"],
                total_positions=agency["total_positions"],
                total_accounts=agency["total_accounts"],
                total_categories=agency["total_categories"]
            )

            # Create Position nodes and relationships
            for position in positions:
                session.run(
                    """
                    MERGE (p:Position {id: $id})
                    SET p.title = $title,
                        p.agency = $agency_name,
                        p.NumPositions = $num_positions,
                        p.NumAccounts = $num_accounts,
                        p.AgencyAccronym = $agency_name_acc

                    WITH p
                    MATCH (a:Agency {name: $agency_name})
                    MERGE (a)-[:HasPosition]->(p)
                    """,                
                    title=position["title"],
                    num_positions=position["num_positions"],
                    num_accounts=position["num_accounts"],
                    agency_name = agency["name"],
                    agency_name_acc=self._extract_acronym(agency["name"]),
                    id = self._extract_acronym(agency["name"]) + "_" + position["title"]
                )

        driver.close()

    def create_employee_and_relationship(self,tx, employee):
        # Create Employee Node
        tx.run(
            """
            MERGE (e:Employee {id: $ID})
            SET e.agency = $Agency, 
                e.RecordGroup = $RecordGroup, 
                e.DisplayName = $DisplayName, 
                e.email = $EmailAddress, 
                e.alias = $Alias, 
                e.name = $Name, 
                e.role = $Role, 
                e.BeginningDate = $BeginningDate, 
                e.EndingDate = $EndingDate, 
                e.CurrentlyInRole = $CurrentlyInRole, 
                e.InRoleAsActing = $InRoleAsActing
            """,
            **employee
        )

        # Check for Matching Position Node and Create Relationship
        tx.run(
            """
            MATCH (e:Employee {id: $ID}), (p:Position {title: $Role, AgencyAccronym: $Agency})
            MERGE (e)-[r:HasServedAs]->(p)
            SET r.StartDate = $BeginningDate, 
                r.EndDate = $EndingDate, 
                r.EmailAddress = $EmailAddress
            """,
            **employee
        )

    def process_file(self, file_path):
        # Read file (assuming tab-separated values)
        df = pd.read_csv(file_path,encoding="ISO-8859-1")

        df["Beginning Date"].replace("-","/")
        df["Ending Date"].replace("-","/")

        df["Beginning Date"] = df["Beginning Date"].apply(self.standardize_date)
        df["Ending Date"] = df["Ending Date"].apply(self.standardize_date)

        df.rename(columns={
        "ID": "ID",
        "Agency": "Agency",
        "Record Group": "RecordGroup",
        "GAL LOOKUP - Display name": "GalDisplayName",
        "GAL LOOKUP - Email Address": "GalEmailAddress",
        "Alias": "Alias",
        "Name": "Name",
        "Role": "Role",
        "Beginning Date": "BeginningDate",
        "Ending Date": "EndingDate",
        "Currently in the Role?": "CurrentlyInRole",
        "In the role as Acting?": "InRoleAsActing",
        "Notes": "Note"
        }, inplace=True)

        driver = self.driver

        with driver.session() as session:
            for _, row in df.iterrows():
                employee_data = {
                    "ID": row["ID"],
                    "Agency": row["Agency"],
                    "RecordGroup": row["RecordGroup"],
                    "DisplayName": row["GalDisplayName"],
                    "EmailAddress": row["GalEmailAddress"],  # Using first email; can modify for second
                    "Alias": row["Alias"],
                    "Name": row["Name"],
                    "Role": row["Role"],
                    "BeginningDate": row["BeginningDate"],
                    "EndingDate": row["EndingDate"],
                    "CurrentlyInRole": row["CurrentlyInRole"],
                    "InRoleAsActing": row["InRoleAsActing"]
                }

                # Create Employee node & establish relationship with Position node
                session.write_transaction(self.create_employee_and_relationship, employee_data)

    def fetch_employees_with_positions(self):
        driver = self.driver
        query = """
MATCH (e:Employee)-[r:HasServedAs]->(p:Position)
WHERE e.name IS NOT NULL AND NOT e.name = 'NaN'
RETURN e, r, p
        """
        with driver.session() as session:
            result = session.run(query)
            data = []
            for record in result:
                employee = record['e']
                relationship = record['r']
                position = record['p']

                # Extracting the relevant data
                employee_data = {
                    "ID": employee.get("id", ""),
                    "Agency": employee.get("agency", ""),
                    "RecordGroup": employee.get("RecordGroup", ""),
                    "DisplayName": employee.get("DisplayName", ""),
                    "EmailAddress": employee.get("email", ""),
                    "Alias": employee.get("alias", ""),
                    "Name": employee.get("name", ""),
                    "Role": employee.get("role", ""),
                    "BeginningDate": relationship.get("StartDate", ""),
                    "EndingDate": relationship.get("EndDate", ""),
                    "CurrentlyInRole": employee.get("CurrentlyInRole", ""),
                    "InRoleAsActing": employee.get("InRoleAsActing", ""),
                    "PositionTitle": position.get("title", ""),
                    "PositionAgency": position.get("agency", ""),
                }
                data.append(employee_data)
            return data

    def save_to_excel(self, data, output_file="employee_positions.xlsx"):
        # Convert the data to a Pandas DataFrame
        df = pd.DataFrame(data)
        
        # Save the DataFrame to an Excel file
        df.to_excel(output_file, index=False)
        print(f"Data saved to {output_file}")

In [ ]:
import os
folder_path = '2023 Capstone Role Approvals-20250302T205115Z-001/2023 Capstone Role Approvals'
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    # Check if the file has .xls or .xlsx extension
    if filename.endswith(('.xls', '.xlsx')):
        # file_path = filename
        URI = "bolt://localhost:7687"  # Update if using a remote instance
        USERNAME = "neo4j"
        PASSWORD = "infochallenge"  # Replace with your actual password
        node1 = add_node(file_path, URI, USERNAME, PASSWORD)
        agency_node, positions = node1.extract_data()
        node1.create_agency_with_positions(agency_node, positions)

node1.process_file("6. Cleansed List of Capstone Officials_DO NOT SHARE-POST-PUBLISH.csv")

# Fetch data from Neo4j and save to Excel
employee_position_data = node1.fetch_employees_with_positions()
# node1.save_to_excel(employee_position_data)